In [1]:
# General
from datetime import datetime
import random
# Data manipulation
import pandas as pd
import numpy as np
import re
# Requests
import requests
from bs4 import BeautifulSoup
# For decoding and reading base64 image
import base64
import pytesseract
from PIL import Image
from io import BytesIO

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
# pd.reset_option()

enable_proxy = False

In [2]:
def decode_vin(encoded_vin):
    im = Image.open(BytesIO(base64.b64decode(encoded_vin))).resize((400, 50))
    return pytesseract.image_to_string(im)

def getProxies():
    pr_list = []
    counter = 0
    for i, row in df_proxy[df_proxy.raw_ip.isin(success_proxy)].iterrows():
        print(row.ip, counter, df_proxy[df_proxy.raw_ip.isin(success_proxy)].shape[0])
        counter += 1
        proxy = {'https': f'https://{row.ip}'}
        try:
            r = requests.get('https://api.ipify.org/', timeout=2, proxies=proxy, headers=headers)
            if row.ip.startswith(r.text):
                r = requests.get('https://autoplius.lt/skelbimai/volvo-xc60-2-4-l-visureigis-2009-dyzelinas-9742229.html', timeout=2, proxies=proxy, headers=headers)
                pr_list.append(row.ip)
        except:
            continue
    return pr_list


USER_AGENT = "Mozilla/5.0 (Windows NT 5.1; rv:9.0.1) Gecko/20100101 Firefox/9.0.1"
headers = requests.utils.default_headers()
headers.update(
    {
        'User-Agent': USER_AGENT,
    }

)

proxy_list = requests.get('https://raw.githubusercontent.com/clarketm/proxy-list/master/proxy-list.txt')
proxies = re.findall('[0-9]+.[0-9]+.[0-9]+.[0-9]+:[0-9]+.*\n', proxy_list.text)
ip_regex = '[0-9]+.[0-9]+.[0-9]+.[0-9]+:[0-9]+'
df_proxy = pd.DataFrame([{'ip': re.findall(ip_regex, proxy)[0], 'country': proxy.split(' ')[1].split('-')[0], 'anonimity': proxy.split(' ')[1].split('-')[1]} for proxy in proxies])
pool_counter = 0
what_is_my_ip = 'https://api.ipify.org/'

status_proxy = requests.get("https://raw.githubusercontent.com/clarketm/proxy-list/master/proxy-list-status.txt")
success_proxy = [re.sub(':.*', '', proxy) for proxy in status_proxy.text.split('\n') if proxy.endswith('success')]
df_proxy['raw_ip'] = df_proxy.ip.apply(lambda x: re.sub(":.*", '', x))

if enable_proxy:
    tst_success = getProxies() 

### Store all available urls into DataFrame

In [ ]:
sitemap = requests.get('https://autoplius.lt/xml_sitemap/index.xml', headers=headers, timeout=30)
soup = BeautifulSoup(sitemap.text, "html.parser")
ads_sitemap = [tag.text for tag in soup.findAll('loc') if 'ann_list' in tag.text]

product_urls = []
last_updated = []
priority = []
for sitemap_url in ads_sitemap:
    print(sitemap_url)
    sitemap_r = requests.get(sitemap_url, headers=headers, timeout=30)
    links_soup = BeautifulSoup(sitemap_r.text, "html.parser")
    
    product_urls.extend([tag.find("xhtml:link", {'hreflang': 'en'})['href'] for tag in links_soup.findAll('url')][0::4])
    last_updated.extend([tag.find('lastmod').text for tag in links_soup.findAll('url')][0::4])
    priority.extend([tag.find('priority').text for tag in links_soup.findAll('url')][0::4])

df_urls = pd.DataFrame({'url': product_urls, 'last_updated': last_updated, 'ad_priority': priority})
#     sitemaps_links.extend({'url': product_urls, 'last_updated': last_updated, 'ad_priority': priority}])

In [ ]:
df_urls.to_csv('urls_dataset.csv', index=False)

In [4]:
df_urls = pd.read_csv('urls_dataset.csv')

In [3]:
df_urls = pd.read_csv('output.csv', lineterminator='\n')

/home/domantas/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (3,4,6,7,8,9,10,11,12,13,14,15,16,17,18,19,22,23,24,26,27,28,29,30,34,35,38,40,41,42,43,44,45,46,47,48,49,50,53,54,55,56,58,59,60,61,62,64,65,67,68,69,70,74,75,77,81,82,84,85,86,89,91,93,94,95,97) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Scrape each url 

In [31]:
import cv2
import pytesseract
import re
import io

def solve_captcha():
    image = Image.open(io.BytesIO(requests.get('https://en.autoplius.lt/utility/captcha/text').content))
    gray = cv2.cvtColor(np.array(image), cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (3,3), 0)
    thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

    # Morph open to remove noise and invert image
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)
    invert = 255 - opening

    # Perform text extraction
    data = re.sub('[ ]*', '', pytesseract.image_to_string(invert, lang='eng', config='--psm 6')).upper()
    res = re.findall('[a-zA-Z]{4}', data)[0] if len(re.findall('[a-zA-Z]{4}', data)) else 'ABCD'
    return res

In [ ]:
def print_exception(column_name, e):
    print(column_name, e)

'''
inspect_exceptions = True: display exceptions message if something triggers except block
inspect_exceptions = False: ignore exceptions message if something triggers except block
'''
inspect_exceptions = False

df = pd.DataFrame()
error_log = pd.DataFrame()

counter = 0
for i, row in df_urls[(df_urls.scrape_date.isnull()) | (df_urls.breadcrumbs == '[]')].iterrows():
    try:
        if enable_proxy:
            try:
                proxy = {'https': f'https://{random.choice(tst_success)}'}
                r = requests.get(url[0], proxies=proxy, headers=headers, timeout=2)
            except:
                try:
                    r = requests.get(url[0], headers=headers, timeout=2)
                except:
                    continue
        else:
            r = requests.get(row.url, headers=headers, timeout=10)
        if r.status_code == 429:
            captcha_text = solve_captcha()
            print('BLOCKED', counter)
            break
        prod_soup = BeautifulSoup(r.text, "html.parser")
        if counter % 50 == 0:
            print(f"{counter}/{df_urls.shape[0]} | {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} ||| {row.url}")
        try:
            df_urls.loc[i, 'title'] = prod_soup.find(attrs = {'class': 'page-title'}).text.replace('\n', '')
        except Exception as e:
            print_exception('title', e) if inspect_exceptions else ''
            df_urls.loc[i, 'title'] = ''

        try:
            df_urls.loc[i, 'price'] = re.sub('(\n)|([ ]+)', '', prod_soup.find(attrs = {'class': 'price'}).text)
        except Exception as e:
            print_exception('price', e) if inspect_exceptions else ''
            df_urls.loc[i, 'price'] = 'Price is negotiable'

        try:
            df_urls.loc[i, 'phone'] = re.sub('(\n)|([ ]+)', '', prod_soup.find(attrs = {'class': 'seller-phone-number'}).text)
        except Exception as e:
            print_exception('phone', e) if inspect_exceptions else ''
            df_urls.loc[i, 'phone'] = ''

        try:
            df_urls.loc[i, 'description'] = prod_soup.findAll('div', {'class': 'announcement-description'})[0].text
        except Exception as e:
            print_exception('description', e) if inspect_exceptions else ''
            df_urls.loc[i, 'description'] = ''

        try:
            df_urls.loc[i, 'contact_name'] = prod_soup.find('div', {'class': 'seller-contact-name'}).text
        except Exception as e:
            print_exception('contact_name', e) if inspect_exceptions else ''
            df_urls.loc[i, 'contact_name'] = ''

        try:
            df_urls.loc[i, 'contact_location'] = re.sub('(\n)|([ ]{2,})', '', prod_soup.find('div', {'class': 'seller-contact-location'}).text)
        except Exception as e:
            print_exception('contact_location', e) if inspect_exceptions else ''
            df_urls.loc[i, 'contact_location'] = ''

        try:
            df_urls.loc[i, 'article_id'] = re.sub('(\n)|([ ]+)', '', prod_soup.find('li', {'class': 'announcement-id'}).text)
        except Exception as e:
            print_exception('article_id', e) if inspect_exceptions else ''
            df_urls.loc[i, 'article_id'] = ''

        try:
            df_urls.loc[i, 'image'] = str([re.sub('ann_[0-9]', 'ann_3', re.findall("https://autoplius-img.dgn.lt/.*.jpg", tag['style'])[0]) for tag in prod_soup.findAll('div', {'class': 'thumbnail'})])
        except Exception as e:
            print_exception('image', e) if inspect_exceptions else ''
            df_urls.loc[i, 'image'] = ''

        try:
            df_urls.loc[i, 'breadcrumbs'] = str([re.sub('(\n)|([ ]+)', '', tag.text) for tag in prod_soup.findAll('li', {'class': 'crumb'})])
        except Exception as e:
            print_exception('breadcrumbs', e) if inspect_exceptions else ''
            df_urls.loc[i, 'breadcrumbs'] = ''

        df_urls.loc[i, 'scrape_date'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        try:
            params_label = [re.sub('(\n)|([ ]{2,})', '', tag.text) for tag in prod_soup.findAll(attrs = {'class': 'parameter-label'})]
            params_value = [re.sub('(\n)|([ ]{2,})', '', tag.text) for tag in prod_soup.findAll(attrs = {'class': 'parameter-value'})]
            params = list(zip(params_label, params_value))
            for p in params:
                df_urls.loc[i, p[0]] = p[1]

            # Extract VIN number if exists
            if len(re.findall('data:image/png;base64, .*"', r.text)):
                encoded_vin = re.sub('data:image/png;base64, ', '', re.findall('data:image/png;base64, .*"', r.text)[0][:-1])
                df_urls.loc[i, 'VIN number'] = decode_vin(encoded_vin)

        except Exception as e:
            error_log = error_log.append([{'params': e, 'url': row.url}])

        try:
            for section in prod_soup.findAll('div', {'class': 'section'}):
                if len(section.findAll('div', {'class': 'feature-row'})):
                    feature_header = re.sub('(\n)|([ ]{2,})', '', section.find('div', {'class': 'heading'}).text)
                    df_urls.loc[i, feature_header] = [section]

        except Exception as e:
            error_log = error_log.append([{'features': e, 'url': row.url}])
        try:
            df_urls.loc[i, 'memorized'] = re.findall('[0-9]+', prod_soup.findAll('span', {'class': 'bookmark-ico'})[0].parent.text)[0]
        except Exception as e:
            print_exception('memorized', e) if inspect_exceptions else ''
            df_urls.loc[i, 'memorized'] = ''
        try:
            df_urls.loc[i, 'update_timestamp'] = [tag.text for tag in prod_soup.findAll('span', {'class': 'bar-item'}) if 'Updated' in tag.text][0]
        except Exception as e:
            print_exception('update_timestamp', e) if inspect_exceptions else ''
            df_urls.loc[i, 'update_timestamp'] = ''

        counter += 1
        if counter % 5000 == 0: 
            df_urls.to_csv('output.csv', index=False)
    except Exception as e:
        print('Failed', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
        break
        
df_urls.to_csv('output.csv', index=False)

In [55]:
payload = {'continue_code': solve_captcha()}

In [47]:
headers = {
    "Host": "autoplius.lt",
    "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:73.0) Gecko/20100101 Firefox/73.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate, br",
    "Content-Type": "application/x-www-form-urlencoded",
    "Content-Length": "14",
    "Origin": "https://autoplius.lt",
    "Connection": "keep-alive",
    "Referer": "https://autoplius.lt/",
    "Cookie": "saved_searches=5dc6816c23d7a; _ga=GA1.2.55427500.1573290350; __gfp_64b=7lEREYyg5MG0kxCY1KoLZtNa_PlvPnfloQDkkYme4TP.U7; z=15732903615dc681797705d; _gid=GA1.2.860668887.1582004260; PHPSESSID=jrg434mbm4beag2qub01s2mmjg",
    "Upgrade-Insecure-Requests": "1",
    "TE": "Trailers"
}

In [56]:
payload

{'continue_code': 'NAYC'}

In [48]:
r = requests.post('https://autoplius.lt/', headers=headers, data=payload)

In [50]:
r

<Response [429]>

In [17]:
df_urls[(~df_urls.scrape_date.isnull()) & (df_urls.breadcrumbs != '[]')]

,url,last_updated,ad_priority,title,price,phone,description,contact_name,contact_location,article_id,image,breadcrumbs,scrape_date,Date of manufacture,Motorcycle type,Engine,Fuel type,Cooling system type,"Kerb weight, kg",New / used,Features / equipment,memorized,update_timestamp,Price,Export,Categories,Type,Purpose of a boat,Manufacturer,Model,Damage,"Length overall, m","Draft, m","Beam, m",Hull material,"Fuel tank capacity, l",Number of toilets,"Weight, kg",Engine(s) type,Number of engines,Engine model,Size,Layout,Euro standard,Mileage,VIN number,Transmission type,Boom type,"Lifting height, mm","Fork length, mm","Lifting capacity, kg",Work hours,Number of seats,Steering wheel,Gearbox,Driven wheels,Part No.,"Volume, m³",Trailer / semi-trailer,Axles make,Number of axles,"Curb weight, kg","Gross weight, kg","Operating width, m","Range, mm","Lifting moment, kNm",Number of passengers,Engine make,Vehicle type,Wheel type,"Size, """,Bolt pattern,"Spacing between holes, mm","Centerbore diameter, mm","Quantity, units",Condition,"Rim width, """,Season,"Remaining tread, %",Year of manufacture,"Offset (ET), mm",Discounts for long term rental,"Structural height, mm",Fit cars
0,https://en.autoplius.lt/ads/shineray-kita-250c...,2020-01-20T00:00:00+02:00,0.55,"Shineray -kita- 250cc, atv / quad / trikes",1285€,3.70686e+10,\n VISOS KAINOS EURAIS SU P...,"HERADAS, UAB","Kėdainiai, Lithuania",ID:A43643,['https://autoplius-img.dgn.lt/ann_3_38261795/...,"['Classifieds', 'Motociklai', 'Shineray', '-ki...",2020-01-26 17:11:47,2019,ATV / Quad / Trikes,"250 cm³, 16 HP (12kW)",Petrol,Air cooling,160 kg,New,NaN,63,Updated 6 days ago,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://en.autoplius.lt/ads/ducati-supermoto-4...,2020-01-25T00:00:00+02:00,0.55,"Ducati, supermoto",Price is negotiable,3.70608e+10,\n Naudotos motociklų detal...,"GTV SOLUTIONS, UAB","Vilnius, Lithuania",ID:A46186,['https://autoplius-img.dgn.lt/ann_3_3269783/d...,"['Classifieds', 'Dalys', 'Motociklųdalys', 'Du...",2020-01-26 17:11:48,NaN,Supermoto,900 cm³,NaN,NaN,NaN,Used,NaN,4,Updated 1 days ago,Price is negotiable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://en.autoplius.lt/ads/yamaha-r6-superbik...,2020-01-21T00:00:00+02:00,0.55,"Yamaha R6, superbikes",70€,3.707e+10,\n R6 199 Remas su dokum...,"ZIPA, UAB","Pasvalys, Lithuania",ID:A46216,['https://autoplius-img.dgn.lt/ann_3_78834753/...,"['Classifieds', 'Motociklųdalys', 'Yamaha', 'R6']",2020-01-26 17:11:48,NaN,Superbikes,600 cm³,NaN,Liquid cooling,NaN,Used,NaN,5,Updated 5 days ago,NaN,70 €,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://en.autoplius.lt/ads/kawasaki-zx-zx-r-n...,2020-01-21T00:00:00+02:00,0.55,"Kawasaki ZX/ZX-R (Ninja), superbikes",15€,3.707e+10,\n 636 - bagažinė 55 ...,"ZIPA, UAB","Pasvalys, Lithuania",ID:A46316,['https://autoplius-img.dgn.lt/ann_3_79825109/...,"['Classifieds', 'Motociklųdalys', 'Kawasaki', ...",2020-01-26 17:11:49,NaN,Superbikes,636 cm³,NaN,Liquid cooling,NaN,Used,NaN,2,Updated 5 days ago,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://en.autoplius.lt/ads/honda-pantheon-sco...,2019-12-23T00:00:00+02:00,0.55,"Honda Pantheon, scooters / mopeds",Price is negotiable,3.70687e+10,\n Honda Panteon (Foresi

In [16]:
r.status_code

429